# Structural DNA helical parameters from MD trajectory tutorial using BioExcel Building Blocks (biobb)
Based on the [**NAFlex**](https://mmb.irbbarcelona.org/NAFlex) server and in particular in its [**Nucleic Acids Analysis**](https://mmb.irbbarcelona.org/NAFlex/help.php?id=tutorialAnalysisNA) section. 

***
This tutorial aims to illustrate the process of **extracting structural and dynamical properties** from a **DNA MD trajectory helical parameters**, step by step, using the **BioExcel Building Blocks library (biobb)**. The particular example used is the **Drew Dickerson Dodecamer** sequence -CGCGAATTCGCG- (PDB code [1BNA](https://www.rcsb.org/structure/1BNA)). The trajectory used is a  500ns-long MD simulation taken from the [BigNASim](https://mmb.irbbarcelona.org/BIGNASim/) database ([NAFlex_DDD_II](https://mmb.irbbarcelona.org/BIGNASim/getStruc.php?idCode=NAFlex_DDD_II) entry).  
***

## Settings

### Biobb modules used

 - [biobb_dna](https://github.com/bioexcel/biobb_dna): Tools to analyse DNA structures and MD trajectories.
 
### Auxiliary libraries used

* [jupyter](https://jupyter.org/): Free software, open standards, and web services for interactive computing across all programming languages.
* [matplotlib](https://matplotlib.org/): Comprehensive library for creating static, animated, and interactive visualizations in Python.

### Conda Installation & Launch

```console
git clone https://github.com/bioexcel/biobb_wf_dna_helparms.git
cd biobb_wf_dna_helparms
conda env create -f conda_env/environment.yml
conda activate biobb_wf_dna_helparms
jupyter-notebook biobb_wf_dna_helparms/notebooks/biobb_wf_dna_helparms.ipynb
```

***
## Pipeline steps
 1. [Input Parameters](#input)
 2. [Running Curves+ and Canal](#curves)
 3. [Average Helical Parameters](#averages)
     1. [Base Pair Step (Inter Base Pair) Parameters](#avg_bps)
     2. [Base Pair (Intra Base Pair) Parameters](#avg_bp)
     3. [Axis Parameters](#avg_axis)
     4. [Grooves](#avg_grooves)
     5. [Backbone Torsions](#avg_backbone)
 8. [Time Series Helical Parameters](#timeseries)
 8. [Stiffness](#stiffness)
 9. [Bimodality](#bimodality)
 10. [Correlations](#correlations)
     1. [Sequence Correlations: Intra-base pairs](#intrasequencecorrelation)
     2. [Sequence Correlations: Inter-base pair steps](#intersequencecorrelation)
     3. [Helical Parameter Correlations: Intra-base pairs](#intrahelparcorrelation)
     4. [Helical Parameter Correlations: Inter-base pair steps](#interhelparcorrelation)
     5. [Neighboring steps Correlations: Intra-base pairs](#intrabasepaircorrelation)
     6. [Neighboring steps Correlations: Inter-base pair steps](#interbasepaircorrelation)
 11. [Questions & Comments](#questions)
 
***
<img src="https://bioexcel.eu/wp-content/uploads/2019/04/Bioexcell_logo_1080px_transp.png" alt="Bioexcel2 logo"
	title="Bioexcel2 logo" width="400" />
***


## Initializing colab
The two cells below are used only in case this notebook is executed via **Google Colab**. Take into account that, for running conda on **Google Colab**, the **condacolab** library must be installed. As [explained here](https://pypi.org/project/condacolab/), the installation requires a **kernel restart**, so when running this notebook in **Google Colab**, don't run all cells until this **installation** is properly **finished** and the **kernel** has **restarted**.

In [ ]:
# Only executed when using google colab
import sys
if 'google.colab' in sys.modules:
  import subprocess
  from pathlib import Path
  try:
    subprocess.run(["conda", "-V"], check=True)
  except FileNotFoundError:
    subprocess.run([sys.executable, "-m", "pip", "install", "condacolab"], check=True)
    import condacolab
    condacolab.install()
    # Clone repository
    repo_URL = "https://github.com/bioexcel/biobb_wf_dna_helparms.git"
    repo_name = Path(repo_URL).name.split('.')[0]
    if not Path(repo_name).exists():
      subprocess.run(["conda", "install", "-y", "git"], check=True)
      subprocess.run(["git", "clone", repo_URL], check=True)
      print("⏬ Repository properly cloned.")
    # Install environment
    print("⏳ Creating environment...")
    env_file_path = f"{repo_name}/conda_env/environment.yml"
    subprocess.run(["conda", "env", "update", "-n", "base", "-f", env_file_path], check=True)
    print("👍 Conda environment successfully created and updated.")

In [ ]:
# Enable widgets for colab
if 'google.colab' in sys.modules:
  from google.colab import output
  output.enable_custom_widget_manager()
  # Change working dir
  import os
  os.chdir("biobb_wf_dna_helparms/biobb_wf_dna_helparms/notebooks")
  print(f"📂 New working directory: {os.getcwd()}")

<a id="input"></a>
## Input parameters
**Input parameters** needed:
 - **seq**: Sequence of the DNA structure (e.g. CGCGAATTCGCG)
 - **seq_comp**: Complementary sequence of the given DNA structure (e.g. CGCGAATTCGCG)
 
 - **traj**: Trajectory for a 500ns Drew Dickerson Dodecamer MD simulation (taken from [BigNASim](https://mmb.irbbarcelona.org/BIGNASim/)) 
 - **top**: Associated topology for the MD trajectory 

In [ ]:
# Auxiliary libraries

import os
import shutil
import glob
from pathlib import Path, PurePath
import zipfile
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import Image
import ipywidgets


In [ ]:
# Input parameters
seq = "CGCGAATTCGCG"
seq_comp = "CGCGAATTCGCG"

traj = "TRAJ/structure.stripped.nc"
top = "TRAJ/structure.stripped.top"

# Auxiliary lists
grooves = ('majd','majw','mind','minw')
axis_base_pairs = ('inclin','tip','xdisp','ydisp')
base_pair = ('shear','stretch','stagger','buckle','propel','opening')
base_pair_step = ('rise','roll','twist','shift','slide','tilt')
backbone_torsions = ('alphaC', 'alphaW', 'betaC', 'betaW', 'gammaC', 'gammaW', 'deltaC', 'deltaW', \
'epsilC', 'epsilW', 'zetaC', 'zetaW', 'chiC', 'chiW', 'phaseC', 'phaseW')

<a id="curves"></a>
## Running Curves+ and Canal

***
**Curves+** program and its associated **Canal** tool allow us to extract **helical parameters** from a **DNA MD simulation**.

**Curves+** is a **nucleic acid conformational analysis program** which provides both **helical** and **backbone parameters**, including a curvilinear axis and parameters relating the position of the bases to this axis. It additionally provides a full analysis of **groove widths** and **depths**. **Curves+** can also be used to analyse **molecular dynamics trajectories**. With the help of the accompanying program **Canal**, it is possible to produce a variety of graphical output including parameter variations along a given structure and **time series** or **histograms** of parameter variations during dynamics.

**Conformational analysis of nucleic acids revisited: Curves+**<br>
*R. Lavery, M. Moakher, J. H. Maddocks, D. Petkeviciute, K. Zakrzewska*<br>
***Nucleic Acids Research, Volume 37, Issue 17, 1 September 2009, Pages 5917–5929***<br>
https://doi.org/10.1093/nar/gkp608


**CURVES+ web server for analyzing and visualizing the helical, backbone and groove parameters of nucleic acid structure.**<br>
*C. Blanchet, M. Pasi, K. Zakrzewska, R. Lavery*<br>
***Nucleic Acids Research, Volume 39, Issue suppl_2, 1 July 2011, Pages W68–W73***<br>
https://doi.org/10.1093/nar/gkr316
http://curvesplus.bsc.es

***
**Building Blocks** used:
- [curves](https://biobb-dna.readthedocs.io/en/latest/curvesplus.html#module-curvesplus.biobb_curves) from **biobb_dna.curvesplus.biobb_curves**
- [canal](https://biobb-dna.readthedocs.io/en/latest/curvesplus.html#module-curvesplus.biobb_canal) from **biobb_dna.curvesplus.biobb_canal**
***
The extraction of **helical parameters** is then done in **two steps**:

- [Step 1: Curves+](#curves1): Reading input **MD trajectory** and analysing **helical parameters**.
- [Step 2: Canal](#canal2): Taking **Curves+** output and generating **time series** and/or **histograms** of parameter variations during dynamics.

***

<a id="curves1"></a>
### Step 1: Curves+

**Curves+** program needs a **trajectory** and its associated **topology**, and a couple of **ranges**, informing about the numeration of the two **DNA strands**: s1range and s2range.

<div style="margin-top:10px;padding:15px;background:#b5e0dd"><strong>Important:</strong> Depending on the operating system used, the cell below can return an error about a missing <strong>.curvesplus</strong> folder. In this case, please copy the <strong>.curvesplus</strong> folder provided in the <strong>repository</strong> and copy it into the <strong>/path/to/anaconda3/envs/biobb_wf_dna_helparms</strong> folder in your computer.</div>

In [ ]:
from biobb_dna.curvesplus.biobb_curves import biobb_curves

curves_out_lis = "curves.out.lis"
curves_out_cda = "curves.out.cda"

prop = {
    's1range' : '1:12',
    's2range' : '24:13',
    # uncomment when running in google colab
    # 'stdlib_path': '.curvesplus/standard'
}

biobb_curves(
    input_struc_path=traj,
    input_top_path=top,
    output_lis_path=curves_out_lis,
    output_cda_path=curves_out_cda,
    properties=prop
)

<a id="canal2"></a>
### Step 2: Canal

**Canal** program needs the output of the previous **Curves+** execution, and is able to produce **time series** (series property) and **histograms** (histo property) for the **parameter variations** during dynamics.

In [ ]:
from biobb_dna.curvesplus.biobb_canal import biobb_canal

canal_out = "canal.out.zip"

prop = {
    'series' : True,
    'histo' : True
}

biobb_canal(
    input_cda_file=curves_out_cda,
    input_lis_file=curves_out_lis,
    output_zip_path=canal_out,
    properties=prop
)

### Extracting Canal results in a temporary folder

In [ ]:
canal_dir = "canal_out"

if Path(canal_dir).exists(): shutil.rmtree(canal_dir) 
os.mkdir(canal_dir)

with zipfile.ZipFile(canal_out, 'r') as zip_ref:
    zip_ref.extractall(canal_dir)

<a id="averages"></a>

## Extracting Average Helical Parameters

**Average helical parameter** values can be computed from the output of **Curves+/Canal** execution. 

The **helical parameters** can be divided in 5 main blocks:

- [Helical Base Pair Step (Inter-Base Pair) Helical Parameters](#avg_bps)
- [Helical Base Pair (Intra-Base Pair) Helical Parameters](#avg_bp)
- [Axis Base Pair](#avg_axis)
- [Grooves](#avg_grooves)
- [Backbone Torsions](#avg_backbone)


<a id="avg_bps"></a>

### Helical Base Pair Step (Inter Base Pair) Parameters

**Translational (Shift, Slide, Rise)** and **rotational (Tilt, Roll, Twist)** parameters related to a **dinucleotide Inter-Base Pair** (Base Pair Step).

- **Shift**: Translation around the X-axis.
- **Slide**: Translation around the Y-axis.
- **Rise**: Translation around the Z-axis.
- **Tilt**: Rotation around the X-axis.
- **Roll**: Rotation around the Y-axis.
- **Twist**: Rotation around the Z-axis.

***
<img src="https://mmb.irbbarcelona.org/NAFlex/images/helicalParamsBPS.png" alt="Helical Base Pair Step Parameters"
	title="Helical Base Pair Step Parameters" width="400" />
***
**Building Block** used:
- [helparaverages](https://biobb-dna.readthedocs.io/en/latest/dna.html#module-dna.dna_averages) from **biobb_dna.dna.dna_averages** 
***

#### Extracting a particular Helical Parameter: Rise

In [ ]:
from biobb_dna.dna.dna_averages import dna_averages

helpar = 'rise'

input_file_path = "canal_out/canal_output" + "_" + helpar + ".ser"
output_averages_csv_path= helpar+'.averages.csv'
output_averages_jpg_path= helpar+'.averages.jpg'

prop = {
    'helpar_name': helpar,
    'sequence': seq
}

dna_averages(
    input_ser_path=input_file_path,
    output_csv_path=output_averages_csv_path,
    output_jpg_path=output_averages_jpg_path,
    properties=prop)

#### Showing the calculated average values for Rise helical parameter

In [ ]:
output_averages_csv_path= helpar+'.averages.csv'
df = pd.read_csv(output_averages_csv_path)
df

#### Plotting the average values for Rise helical parameter 

In [ ]:
Image(filename=output_averages_jpg_path,width = 600)

#### Computing average values from all base-pair step parameters

In [ ]:
from biobb_dna.dna.dna_averages import dna_averages

for helpar in base_pair_step:

    input_file_path = "canal_out/canal_output" + "_" + helpar + ".ser"
    output_averages_csv_path= helpar+'.averages.csv'
    output_averages_jpg_path= helpar+'.averages.jpg'

    prop = {
        'helpar_name': helpar,
        'sequence': seq
    }

    dna_averages(
        input_ser_path=input_file_path,
        output_csv_path=output_averages_csv_path,
        output_jpg_path=output_averages_jpg_path,
        properties=prop)

#### Showing the calculated average values for all base-pair step helical parameters

In [ ]:
for helpar in base_pair_step:
    output_averages_csv_path= helpar+'.averages.csv'
    df = pd.read_csv(output_averages_csv_path)
    print("Helical Parameter: " + helpar)
    print(df)
    print("---------\n")

#### Plotting the average values for all base-pair step helical parameters 

In [ ]:
images = []
for helpar in base_pair_step:
    images.append(helpar + '.averages.jpg')

f, axarr = plt.subplots(2, 3, figsize=(40, 20))

for i, image in enumerate(images):
    y = i%3
    x = int(i/3)

    img = mpimg.imread(image)

    axarr[x,y].imshow(img, aspect='auto')
    axarr[x,y].axis('off')

plt.show()


<a id="avg_bp"></a>
### Helical Base Pair (Intra Base Pair) Parameters

**Translational (Shear, Stretch, Stagger)** and **rotational (Buckle, Propeller, Opening)** parameters related to a **dinucleotide Intra-Base Pair**.

- **Shear**: Translation around the X-axis.
- **Stretch**: Translation around the Y-axis.
- **Stagger**: Translation around the Z-axis.
- **Buckle**: Rotation around the X-axis.
- **Propeller**: Rotation around the Y-axis.
- **Opening**: Rotation around the Z-axis.

***
<img src="https://mmb.irbbarcelona.org/NAFlex/images/helicalParamsBP.png" alt="Helical Base Pair Parameters"
	title="Helical Base Pair Parameters" width="400" />
***
**Building Block** used:
- [helparaverages](https://biobb-dna.readthedocs.io/en/latest/dna.html#module-dna.dna_averages) from **biobb_dna.dna.dna_averages** 
***

#### Computing average values from all base-pair parameters

In [ ]:
from biobb_dna.dna.dna_averages import dna_averages

for helpar in base_pair:

    #input_file_path = canal_out + "_" + helpar + ".ser"
    input_file_path = "canal_out/canal_output" + "_" + helpar + ".ser"
    output_averages_csv_path= helpar+'.averages.csv'
    output_averages_jpg_path= helpar+'.averages.jpg'

    prop = {
        'helpar_name': helpar,
        'sequence': seq
    }

    dna_averages(
        input_ser_path=input_file_path,
        output_csv_path=output_averages_csv_path,
        output_jpg_path=output_averages_jpg_path,
        properties=prop)

#### Showing the calculated average values for all base-pair helical parameters

In [ ]:
for helpar in base_pair:
    output_averages_csv_path= helpar+'.averages.csv'
    df = pd.read_csv(output_averages_csv_path)
    print("Helical Parameter: " + helpar)
    print(df)
    print("---------\n")

#### Plotting the average values for all base-pair helical parameters 

In [ ]:
images = []
for helpar in base_pair:
    images.append(helpar + '.averages.jpg')

f, axarr = plt.subplots(2, 3, figsize=(40, 20))

for i, image in enumerate(images):
    y = i%3
    x = int(i/3)

    img = mpimg.imread(image)

    axarr[x,y].imshow(img, aspect='auto')
    axarr[x,y].axis('off')

plt.show()

<a id="avg_axis"></a>
### Axis Base Pair Parameters

***
**Translational (x/y-displacement)** and **rotational (inclination, tip)** parameters related to a dinucleotide Base Pair.

- **X-displacement**: Translation around the X-axis.
- **Y-displacement**: Translation around the Y-axis.
- **Inclination**: Rotation around the X-axis.
- **Tip**: Rotation around the Y-axis.
***
<img src="https://mmb.irbbarcelona.org/NAFlex/images/axis-bp.png" alt="Axis Base Pair Parameters"
	title="Axis Base Pair Parameters" width="200" />
***
**Building Block** used:
- [helparaverages](https://biobb-dna.readthedocs.io/en/latest/dna.html#module-dna.dna_averages) from **biobb_dna.dna.averages** 
***

#### Computing average values from all Axis base-pair parameters

In [ ]:
from biobb_dna.dna.dna_averages import dna_averages

for helpar in axis_base_pairs:

    #input_file_path = canal_out + "_" + helpar + ".ser"
    input_file_path = "canal_out/canal_output" + "_" + helpar + ".ser"
    output_averages_csv_path= helpar+'.averages.csv'
    output_averages_jpg_path= helpar+'.averages.jpg'

    prop = {
        'helpar_name': helpar,
        'sequence': seq,
#        'seqpos': [4,3]
    }

    dna_averages(
        input_ser_path=input_file_path,
        output_csv_path=output_averages_csv_path,
        output_jpg_path=output_averages_jpg_path,
        properties=prop)

#### Showing the calculated average values for all Axis base-pair helical parameters

In [ ]:
for helpar in axis_base_pairs:
    output_averages_csv_path= helpar+'.averages.csv'
    df = pd.read_csv(output_averages_csv_path)
    print("Helical Parameter: " + helpar)
    print(df)
    print("---------\n")

#### Plotting the average values for all Axis base-pair helical parameters 

In [ ]:
images = []
for helpar in axis_base_pairs:
    images.append(helpar + '.averages.jpg')

f, axarr = plt.subplots(2, 2, figsize=(30, 20))

for i, image in enumerate(images):
    y = i%2
    x = int(i/2)

    img = mpimg.imread(image)

    axarr[x,y].imshow(img, aspect='auto')
    axarr[x,y].axis('off')

plt.show()

<a id="avg_grooves"></a>
### Grooves

***
Nucleic Acid Structure's strand backbones appear closer together on one side of the helix than on the other. This creates a **Major groove** (where backbones are far apart) and a **Minor groove** (where backbones are close together). **Depth and width** of these grooves can be mesured giving information about the different conformations that the nucleic acid structure can achieve.

- **Major Groove Width**.
- **Major Groove Depth**.
- **Minor Groove Width**.
- **Minor Groove Depth**.

***
<img src="https://mmb.irbbarcelona.org//NAFlex2/images/DnaMajorMinorGroove.gif" alt="Grooves Parameters"
	title="Grooves Parameters" width="200" />
***
**Building Block** used:
- [helparaverages](https://biobb-dna.readthedocs.io/en/latest/dna.html#module-dna.dna_averages) from **biobb_dna.dna.dna_averages**
***

#### Computing average values from all Grooves parameters

In [ ]:
from biobb_dna.dna.dna_averages import dna_averages

for helpar in grooves:

    #input_file_path = canal_out + "_" + helpar + ".ser"
    input_file_path = "canal_out/canal_output" + "_" + helpar + ".ser"
    output_averages_csv_path= helpar+'.averages.csv'
    output_averages_jpg_path= helpar+'.averages.jpg'

    prop = {
        'helpar_name': helpar,
        'sequence': seq
    }

    dna_averages(
        input_ser_path=input_file_path,
        output_csv_path=output_averages_csv_path,
        output_jpg_path=output_averages_jpg_path,
        properties=prop)

#### Showing the calculated average values for all Grooves parameters

In [ ]:
for helpar in grooves:
    output_averages_csv_path= helpar+'.averages.csv'
    df = pd.read_csv(output_averages_csv_path)
    print("Helical Parameter: " + helpar)
    print(df)
    print("---------\n")

#### Plotting the average values for all Grooves helical parameters 

In [ ]:
images = []
for helpar in grooves:
    images.append(helpar + '.averages.jpg')

f, axarr = plt.subplots(2, 2, figsize=(30, 20))

for i, image in enumerate(images):
    y = i%2
    x = int(i/2)

    img = mpimg.imread(image)

    axarr[x,y].imshow(img, aspect='auto')
    axarr[x,y].axis('off')

plt.show()

<a id="avg_backbone"></a>
### Backbone Torsions

***

 The three major elements of flexibility in the **backbone** are:

- **[Sugar Puckering](#puckering)**:

    Sugar Puckering annotation is done by dividing the pseudo-rotational circle in four equivalent sections:<br><br>
    - ***North***: 315:45º
    - ***East***: 45:135º
    - ***South***: 135:225º
    - ***West***: 225:315º
    
 These four conformations are those dominating sugar conformational space, in agreement with all available experimental data.


- **[Canonical Alpha/Gamma](#alphagamma)**:

    Rotations around **α/γ torsions** generate non-canonical local conformations leading to a reduced twist and they have been reported as being important in the formation of several protein-DNA complexes. 
    
    
- **[BI/BII Population](#bIbII)**:

    The concerted rotation around **ζ/ε torsions** generates two major conformers: **BI and BII**, which are experimentally known to co-exist in a ratio around 80%:20% (BI:BII) in B-DNA.


<table><tr style="background-color: #FFFFFF"><td>
    <img src="https://mmb.irbbarcelona.org/NAFlex/images/Puckering2.png" alt="Sugar Puckering"
	title="Sugar Puckering" width="300" /> 
</td><td>
    <img src="https://mmb.irbbarcelona.org/NAFlex/images/AlphaGamma.png" alt="Canonical Alpha/Gamma"
title="Canonical Alpha/Gamma" width="300" />
</td><td>
    <img src="https://mmb.irbbarcelona.org/NAFlex/images/BI-BII.png" alt="BI/BII population"
title="BI/BII population" width="300" />
</td></tr>
<tr ><td>
    <p style="text-align: center; font-weight: bold">Sugar Puckering</p>
</td>
<td>
    <p style="text-align: center; font-weight: bold">Canonical Alpha/Gamma</p>
</td>
<td>
    <p style="text-align: center; font-weight: bold">BI/BII population</p>
</td></tr>
</table>
    
***
**Building Blocks** used:
- [puckering](https://biobb-dna.readthedocs.io/en/latest/backbone.html#module-backbone.puckering) from **biobb_dna.backbone.puckering** 
- [canonicalag](https://biobb-dna.readthedocs.io/en/latest/backbone.html#module-backbone.canonicalag) from **biobb_dna.backbone.canonicalag**
- [bipopulations](https://biobb-dna.readthedocs.io/en/latest/backbone.html#module-backbone.bipopulations) from **biobb_dna.backbone.bipopulations**
***

<a id="puckering"></a>

#### Sugar Puckering

##### Computing average values

In [ ]:
from biobb_dna.backbone.puckering import puckering

canal_phaseC = "canal_out/canal_output_phaseC.ser"
canal_phaseW = "canal_out/canal_output_phaseW.ser"

output_puckering_csv_path = 'puckering.averages.csv'
output_puckering_jpg_path = 'puckering.averages.jpg'

prop = {
    'sequence': seq
}

puckering(
    input_phaseC_path=canal_phaseC,
    input_phaseW_path=canal_phaseW,
    output_csv_path=output_puckering_csv_path,
    output_jpg_path=output_puckering_jpg_path,
    properties=prop)

##### Showing the calculated average values

In [ ]:
df = pd.read_csv(output_puckering_csv_path)
df

##### Plotting the average values 

In [ ]:
Image(filename=output_puckering_jpg_path,width = 600)

<a id="alphagamma"></a>
#### Canonical Alpha/Gamma

##### Computing average values

In [ ]:
from biobb_dna.backbone.canonicalag import canonicalag

canal_alphaC = "canal_out/canal_output_alphaC.ser"
canal_alphaW = "canal_out/canal_output_alphaW.ser"
canal_gammaC = "canal_out/canal_output_gammaC.ser"
canal_gammaW = "canal_out/canal_output_gammaW.ser"

output_alphagamma_csv_path = 'alphagamma.averages.csv'
output_alphagamma_jpg_path = 'alphagamma.averages.jpg'

prop = {
    'sequence': seq
}

canonicalag(
    input_alphaC_path=canal_alphaC,
    input_alphaW_path=canal_alphaW,
    input_gammaC_path=canal_gammaC,
    input_gammaW_path=canal_gammaW,
    output_csv_path=output_alphagamma_csv_path,
    output_jpg_path=output_alphagamma_jpg_path,
    properties=prop)

##### Showing the calculated average values

In [ ]:
df = pd.read_csv(output_alphagamma_csv_path)
df

##### Plotting the average values 

In [ ]:
Image(filename=output_alphagamma_jpg_path,width = 600)

<a id="bIbII"></a>
#### BI/BII Population

##### Computing average values

In [ ]:
from biobb_dna.backbone.bipopulations import bipopulations

canal_epsilC = "canal_out/canal_output_epsilC.ser"
canal_epsilW = "canal_out/canal_output_epsilW.ser"
canal_zetaC = "canal_out/canal_output_zetaC.ser"
canal_zetaW = "canal_out/canal_output_zetaW.ser"

output_bIbII_csv_path = 'bIbII.averages.csv'
output_bIbII_jpg_path = 'bIbII.averages.jpg'

prop = {
    'sequence': seq
}

bipopulations(
    input_epsilC_path=canal_epsilC,
    input_epsilW_path=canal_epsilW,
    input_zetaC_path=canal_zetaC,
    input_zetaW_path=canal_zetaW,
    output_csv_path=output_bIbII_csv_path,
    output_jpg_path=output_bIbII_jpg_path,
    properties=prop)


##### Showing the calculated average values

In [ ]:
df = pd.read_csv(output_bIbII_csv_path)
df

##### Plotting the average values 

In [ ]:
Image(filename=output_bIbII_jpg_path,width = 600)

<a id="timeseries"></a>
## Extracting Time series Helical Parameters

**Time series** values for the set of **helical parameters** can be also extracted from the output of **Curves+/Canal** execution on **Molecular Dynamics Trajectories**. The **helical parameters** can be divided in the same 5 main blocks previously introduced:

- Helical Base Pair Step (Inter-Base Pair) Helical Parameters
- Helical Base Pair (Intra-Base Pair) Helical Parameters
- Axis Base Pair
- Grooves
- Backbone Torsions

***
**Building Block** used:
- [helpartimeseries](https://biobb-dna.readthedocs.io/en/latest/dna.html#module-dna.dna_timeseries) from **biobb_dna.dna.dna_timeseries**
***

### Extracting a particular Helical Parameter

**Time series** values can be extracted from any of the **helical parameters** previously introduced. To illustrate the steps needed, the **base-pair step helical parameter** ***Twist*** has been selected. Please note that computing the **time series** values for a different **helical parameter** just requires modifying the ***helpar*** variable from the next cell.  

In [ ]:
from biobb_dna.dna.dna_timeseries import dna_timeseries

# Modify the next variable to extract time series values for a different helical parameter
# Possible values are: 
    # Base Pair Step (Inter Base Pair) Helical Parameters: shift, slide, rise, tilt, roll, twist 
    # Base Pair (Intra Base Pair) Helical Parameters: shear, stretch, stagger, buckle, propeller, opening
    # Axis Parameters: inclin, tip, xdisp, ydisp
    # Backbone Torsions Parameters: alphaC, alphaW, betaC, betaW, gammaC, gammaW, deltaC, deltaW,
    #                                epsilC, epsilW, zetaC, zetaW, chiC, chiW, phaseC, phaseW
    # Grooves: mind, minw, majd, majw

helpar = "twist" # Modify this variable to extract time series values for a different helical parameter

input_file_path = "canal_out/canal_output" + "_" + helpar + ".ser"
output_timeseries_file_path = helpar + '.timeseries.zip'

prop = {
    'helpar_name': helpar,
    'sequence': seq
}

dna_timeseries(
    input_ser_path=input_file_path,
    output_zip_path=output_timeseries_file_path,
    properties=prop)

### Extracting time series results for the selected helical parameter in a temporary folder

In [ ]:
timeseries_dir = "timeseries"

if Path(timeseries_dir).exists(): shutil.rmtree(timeseries_dir) 
os.mkdir(timeseries_dir)

with zipfile.ZipFile(output_timeseries_file_path, 'r') as zip_ref:
    zip_ref.extractall(timeseries_dir)

### Finding out all the possible nucleotide / base / base-pair / base-pair steps

Discover all the possible **nucleotide / base / base-pair / base-pair steps** from the **sequence**. The unit will depend on the helical parameter being studied.  
**Select one of them** to study the **time series** values of the **helical parameter** along the **simulation**.

In [ ]:
helpartimesfiles = glob.glob(timeseries_dir + "/*series*.csv") 

helpartimes = []
for file in helpartimesfiles:
    new_string = file.replace(timeseries_dir + "/series_" + helpar + "_", "")
    new_string = new_string.replace(".csv" , "")
    helpartimes.append(new_string)
    
timesel = ipywidgets.Dropdown(
    options=helpartimes,
    description='Sel. BPS:',
    disabled=False,
)
display(timesel)

### Showing the time series values for the selected unit

In [ ]:
file_ser = timeseries_dir + "/series_" + helpar + "_" + timesel.value + ".csv"

df = pd.read_csv(file_ser)
df

In [ ]:
file_hist = timeseries_dir + "/hist_" + helpar + "_" + timesel.value + ".csv"

df = pd.read_csv(file_hist)
df

### Plotting the time series values for the selected base-pair step 

In [ ]:
file_ser = timeseries_dir + "/series_" + helpar + "_" + timesel.value + ".jpg"
file_hist = timeseries_dir + "/hist_" + helpar + "_" + timesel.value + ".jpg"

images = []

images.append(file_ser)
images.append(file_hist)

f, axarr = plt.subplots(1, 2, figsize=(50, 15))

for i, image in enumerate(images):
    img = mpimg.imread(image)

    axarr[i].imshow(img, aspect='auto')
    axarr[i].axis('off')

plt.show()

### Computing timeseries for all base-pair step parameters 

In [ ]:
from biobb_dna.dna.dna_timeseries import dna_timeseries

output_timeseries_bps_file_paths = {}
for helpar in base_pair_step:

    input_file_path = "canal_out/canal_output" + "_" + helpar + ".ser"
    output_timeseries_bps_file_paths[helpar] = helpar + '.timeseries.zip'

    prop = {
        'helpar_name': helpar,
        'sequence': seq
    }

    dna_timeseries(
        input_ser_path=input_file_path,
        output_zip_path=output_timeseries_bps_file_paths[helpar],
        properties=prop)

In [ ]:
#if Path(timeseries_dir).exists(): shutil.rmtree(timeseries_dir) 
#os.mkdir(timeseries_dir)

for timeseries_zipfile in output_timeseries_bps_file_paths.values():
    with zipfile.ZipFile(timeseries_zipfile, 'r') as zip_ref:
        zip_ref.extractall(timeseries_dir)

### Computing timeseries for all base-pair parameters 

In [ ]:
from biobb_dna.dna.dna_timeseries import dna_timeseries

output_timeseries_bp_file_paths = {}
for helpar in base_pair:

    input_file_path = "canal_out/canal_output" + "_" + helpar + ".ser"
    output_timeseries_bp_file_paths[helpar] = helpar + '.timeseries.zip'

    prop = {
        'helpar_name': helpar,
        'sequence': seq
    }

    dna_timeseries(
        input_ser_path=input_file_path,
        output_zip_path=output_timeseries_bp_file_paths[helpar],
        properties=prop)

In [ ]:
#if Path(timeseries_dir).exists(): shutil.rmtree(timeseries_dir) 
#os.mkdir(timeseries_dir)

for timeseries_zipfile in output_timeseries_bp_file_paths.values():
    with zipfile.ZipFile(timeseries_zipfile, 'r') as zip_ref:
        zip_ref.extractall(timeseries_dir)

### Computing timeseries for all axis parameters 

In [ ]:
from biobb_dna.dna.dna_timeseries import dna_timeseries

output_timeseries_bp_file_paths = {}
for helpar in axis_base_pairs:

    input_file_path = "canal_out/canal_output" + "_" + helpar + ".ser"
    output_timeseries_bp_file_paths[helpar] = helpar + '.timeseries.zip'

    prop = {
        'helpar_name': helpar,
        'sequence': seq
    }

    dna_timeseries(
        input_ser_path=input_file_path,
        output_zip_path=output_timeseries_bp_file_paths[helpar],
        properties=prop)

In [ ]:
for timeseries_zipfile in output_timeseries_bp_file_paths.values():
    with zipfile.ZipFile(timeseries_zipfile, 'r') as zip_ref:
        zip_ref.extractall(timeseries_dir)

### Computing timeseries for all grooves parameters 

In [ ]:
from biobb_dna.dna.dna_timeseries import dna_timeseries

output_timeseries_bp_file_paths = {}
for helpar in grooves:

    input_file_path = "canal_out/canal_output" + "_" + helpar + ".ser"
    output_timeseries_bp_file_paths[helpar] = helpar + '.timeseries.zip'

    prop = {
        'helpar_name': helpar,
        'sequence': seq
    }

    dna_timeseries(
        input_ser_path=input_file_path,
        output_zip_path=output_timeseries_bp_file_paths[helpar],
        properties=prop)

In [ ]:
for timeseries_zipfile in output_timeseries_bp_file_paths.values():
    with zipfile.ZipFile(timeseries_zipfile, 'r') as zip_ref:
        zip_ref.extractall(timeseries_dir)

### Computing timeseries for all backbone torsions parameters 

In [ ]:
from biobb_dna.dna.dna_timeseries import dna_timeseries

output_timeseries_bp_file_paths = {}
for helpar in backbone_torsions:

    input_file_path = "canal_out/canal_output" + "_" + helpar + ".ser"
    output_timeseries_bp_file_paths[helpar] = helpar + '.timeseries.zip'

    prop = {
        'helpar_name': helpar,
        'sequence': seq
    }

    dna_timeseries(
        input_ser_path=input_file_path,
        output_zip_path=output_timeseries_bp_file_paths[helpar],
        properties=prop)

In [ ]:
for timeseries_zipfile in output_timeseries_bp_file_paths.values():
    with zipfile.ZipFile(timeseries_zipfile, 'r') as zip_ref:
        zip_ref.extractall(timeseries_dir)

<a id="stiffness"></a>
## Stiffness

**Molecular stiffness** is an **elastic force constant** associated with **helical deformation** at the **base pair step** level and is determined by inversion of the covariance matrix in helical space, which yields **stiffness matrices** whose diagonal elements provide the **stiffness constants** associated with **pure rotational** (twist, roll and tilt) and **translational** (rise, shift and slide) **deformations** within the given step.

***
<img src="https://mmb.irbbarcelona.org/NAFlex2/images/StiffnessMatrix.png" alt="Stiffness Matrix"
	title="Stiffness Matrix" width="500" />
***
**Building Blocks** used:
- [averagestiffness](https://biobb-dna.readthedocs.io/en/latest/stiffness.html#module-stiffness.average_stiffness) from **biobb_dna.stiffness.average_stiffness** 
- [bpstiffness](https://biobb-dna.readthedocs.io/en/latest/stiffness.html#module-stiffness.basepair_stiffness) from **biobb_dna.stiffness.basepair_stiffness** 
***


In [ ]:
from biobb_dna.stiffness.average_stiffness import average_stiffness

helpar = "twist" # Modify this variable to extract time series values for a different helical parameter

input_file_path = "canal_out/canal_output" + "_" + helpar + ".ser"
output_stiffness_csv_path = helpar + '.stiffness.csv'
output_stiffness_jpg_path = helpar + '.stiffness.jpg'

prop = { 
    'sequence' : seq
}

average_stiffness(
    input_ser_path=input_file_path,
    output_csv_path=output_stiffness_csv_path,
    output_jpg_path=output_stiffness_jpg_path,
    properties=prop)


In [ ]:
df = pd.read_csv(output_stiffness_csv_path)
df

In [ ]:
Image(filename=output_stiffness_jpg_path,width = 600)

In [ ]:
from biobb_dna.stiffness.basepair_stiffness import basepair_stiffness

timeseries_shift = timeseries_dir+"/series_shift_" + timesel.value + ".csv"
timeseries_slide = timeseries_dir+"/series_slide_" + timesel.value + ".csv"
timeseries_rise = timeseries_dir+"/series_rise_" + timesel.value + ".csv"
timeseries_tilt = timeseries_dir+"/series_tilt_" + timesel.value + ".csv"
timeseries_roll = timeseries_dir+"/series_roll_" + timesel.value + ".csv"
timeseries_twist = timeseries_dir+"/series_twist_" + timesel.value + ".csv"

output_stiffness_bps_csv_path = "stiffness_bps.csv"
output_stiffness_bps_jpg_path = "stiffness_bps.jpg"

basepair_stiffness(
    input_filename_shift=timeseries_shift,
    input_filename_slide=timeseries_slide,
    input_filename_rise=timeseries_rise,
    input_filename_tilt=timeseries_tilt,
    input_filename_roll=timeseries_roll,
    input_filename_twist=timeseries_twist,
    output_csv_path=output_stiffness_bps_csv_path,
    output_jpg_path=output_stiffness_bps_jpg_path)

In [ ]:
df = pd.read_csv(output_stiffness_bps_csv_path)
df

In [ ]:
Image(filename=output_stiffness_bps_jpg_path,width = 600)

<a id="bimodality"></a>
## Bimodality

**Base-pair steps** helical parameters usually follow a normal (Gaussian-like) distribution. However, recent studies observed **bimodal distributions** in some **base-pair steps** for **twist and slide**, highlighting potential caveats on the **harmonic approximation** implicit in **elastic analysis** and mesoscopic models of DNA flexibility.

***
<img src="https://mmb.irbbarcelona.org/BIGNASim/htmlib/help/img/Tut3_GCGA_Twist_plot.png" alt="Twist bimodality"
	title="Twist bimodality" width="600" />
***

**μABC: a systematic microsecond molecular dynamics study of tetranucleotide sequence effects in B-DNA**
*Marco Pasi, John H Maddocks, David Beveridge, Thomas C Bishop, David A Case, Thomas Cheatham 3rd, Pablo D Dans, B Jayaram, Filip Lankas, Charles Laughton, Jonathan Mitchell, Roman Osman, Modesto Orozco, Alberto Pérez, Daiva Petkevičiūtė, Nada Spackova, Jiri Sponer, Krystyna Zakrzewska, Richard Lavery*
***Nucleic Acids Research 2014, Volume 42, Issue 19, Pages 12272-12283***<br>
https://doi.org/10.1093/nar/gku855

**Exploring polymorphisms in B-DNA helical conformations**<br>
*Pablo D Dans, Alberto Pérez, Ignacio Faustino, Richard Lavery, Modesto Orozco*<br>
***Nucleic Acids Research 2012, Volume 40, Issue 21, Pages 10668-10678***<br>
https://doi.org/10.1093/nar/gks884

**A systematic molecular dynamics study of nearest-neighbor effects on base pair and base pair step conformations and fluctuations in B-DNA**<br>
*Lavery R, Zakrzewska K, Beveridge D, Bishop TC, Case DA, Cheatham T, III, Dixit S, Jayaram B, Lankas F, Laughton C, John H Maddocks, Alexis Michon, Roman Osman, Modesto Orozco, Alberto Perez, Tanya Singh, Nada Spackova, Jiri Sponer*<br>
***Nucleic Acids Research 2010, Volume 38, Pages 299–313***<br>
https://doi.org/10.1093/nar/gkp834

***

**Building Block** used:
- [helparbimodality](https://biobb-dna.readthedocs.io/en/latest/dna.html#module-dna.dna_bimodality) from **biobb_dna.dna.bimodality** 
***

In [ ]:
from biobb_dna.dna.dna_bimodality import dna_bimodality

helpar = "twist"
input_csv = timeseries_dir+"/series_"+helpar+"_"+timesel.value+'.csv'
#input_csv = "/Users/hospital/biobb_tutorials/biobb_dna/timeseries"+"/series_"+timesel.value+'.csv' # <-- TO BE REPLACED BY PREVIOUS LINE 

output_bimodality_csv = helpar+'.bimodality.csv'
output_bimodality_jpg = helpar+'.bimodality.jpg'

prop = {
    'max_iter': 500
}
dna_bimodality(
    input_csv_file=input_csv,
    output_csv_path=output_bimodality_csv,
    output_jpg_path=output_bimodality_jpg,
    properties=prop)

In [ ]:
file_hist = timeseries_dir + "/hist_" + helpar + "_" + timesel.value + ".jpg"
file_bi = helpar + ".bimodality.jpg"

images = []

images.append(file_hist)
images.append(file_bi)

f, axarr = plt.subplots(1, 2, figsize=(50, 15))

for i, image in enumerate(images):
    img = mpimg.imread(image)

    axarr[i].imshow(img, aspect='auto')
    axarr[i].axis('off')

plt.show()

<a id="correlations"></a>
## Correlations

Sequence-dependent **correlation movements** have been identified in DNA conformational analysis at the **base pair** and **base pair-step** level. **Trinucleotides** were found to show moderate to high **correlations** in some **intra base pair helical parameter** (e.g. shear-opening, shear-stretch, stagger-buckle). Similarly, some **tetranucleotides** are showing strong **correlations** in their **inter base pair helical parameters** (e.g. shift-tilt, slide-twist, rise-tilt, shift-slide, and shift-twist in RR steps), with **negative correlations** in the shift-slide and roll-twist cases. **Correlations** are also observed in the combination of **inter- and intra-helical parameters** (e.g. shift-opening, rise-buckle, stagger-tilt). **Correlations** analysis can be also extended to **neighboring steps** (e.g. twist in the central YR step of XYRR tetranucleotides with slide in the adjacent RR step). 

- [Sequence Correlations: Intra-base pairs](#intrasequencecorrelation)
- [Sequence Correlations: Inter-base pair steps](#intersequencecorrelation)
- [Helical Parameter Correlations: Intra-base pairs](#intrahelparcorrelation)
- [Helical Parameter Correlations: Inter-base pair steps](#interhelparcorrelation)
- [Neighboring steps Correlations: Intra-base pairs](#intrabasepaircorrelation)
- [Neighboring steps Correlations: Inter-base pair steps](#interbasepaircorrelation)

***
<img src="https://mmb.irbbarcelona.org/NAFlex2/images/rise_correlations.png" alt="Rise correlations"
	title="Rise correlations" width="400" />
***

**The static and dynamic structural heterogeneities of B-DNA: extending Calladine–Dickerson rules**
*Pablo D Dans, Alexandra Balaceanu, Marco Pasi, Alessandro S Patelli, Daiva Petkevičiūtė, Jürgen Walther, Adam Hospital, Genís Bayarri, Richard Lavery, John H Maddocks, Modesto Orozco*
***Nucleic Acids Research 2019, Volume 47, Issue 21, Pages 11090-11102***<br>
https://doi.org/10.1093/nar/gkz905

***

**Building Blocks** used:
- [intrasequencecorrelation](https://biobb-dna.readthedocs.io/en/latest/intrabp_correlations.html#module-intrabp_correlations.intraseqcorr) from **biobb_dna.intrabp_correlations.intraseqcorr** 
- [intersequencecorrelation](https://biobb-dna.readthedocs.io/en/latest/interbp_correlations.html#interbp-correlations-interseqcorr-module) from **biobb_dna.interbp_correlations.interseqcorr** 
- [intrahelparcorrelation](https://biobb-dna.readthedocs.io/en/latest/intrabp_correlations.html#intrabp-correlations-intrahpcorr-module) from **biobb_dna.intrabp_correlations.intrahpcorr** 
- [interhelparcorrelation](https://biobb-dna.readthedocs.io/en/latest/interbp_correlations.html#interbp-correlations-interhpcorr-module) from **biobb_dna.interbp_correlations.interhpcorr** 
- [intrabasepaircorrelation](https://biobb-dna.readthedocs.io/en/latest/intrabp_correlations.html#intrabp-correlations-intrabpcorr-module) from **biobb_dna.intrabp_correlations.intrabpcorr** 
- [interbasepaircorrelation](https://biobb-dna.readthedocs.io/en/latest/interbp_correlations.html#interbp-correlations-interbpcorr-module) from **biobb_dna.interbp_correlations.interbpcorr** 

***

<a id="intrasequencecorrelation"></a>
### Sequence Correlations: Intra-base pairs

In [ ]:
from biobb_dna.intrabp_correlations.intraseqcorr import intraseqcorr

input_file_path = "canal_out/canal_output" + "_" + helpar + ".ser"
output_intrabp_correlation_csv_path = helpar+'.intrabp_correlation.csv'
output_intrabp_correlation_jpg_path = helpar+'.intrabp_correlation.jpg'

prop={
    'sequence' : seq,
#    'helpar_name' : 'Rise'
}

intraseqcorr(
    input_ser_path=input_file_path,
    output_csv_path=output_intrabp_correlation_csv_path,
    output_jpg_path=output_intrabp_correlation_jpg_path,
    properties=prop)

In [ ]:
df = pd.read_csv(output_intrabp_correlation_csv_path)
df

In [ ]:
Image(filename=output_intrabp_correlation_jpg_path,width = 600)

<a id="intersequencecorrelation"></a>
### Sequence Correlations: Inter-base pair steps

In [ ]:
from biobb_dna.interbp_correlations.interseqcorr import interseqcorr

input_file_path = "canal_out/canal_output" + "_" + helpar + ".ser"
output_interbp_correlation_csv_path = helpar+'.interbp_correlation.csv'
output_interbp_correlation_jpg_path = helpar+'.interbp_correlation.jpg'

prop={
    'sequence' : seq,
#    'helpar_name' : 'Rise'
}

interseqcorr(
    input_ser_path=input_file_path,
    output_csv_path=output_interbp_correlation_csv_path,
    output_jpg_path=output_interbp_correlation_jpg_path,
    properties=prop)

In [ ]:
df = pd.read_csv(output_interbp_correlation_csv_path)
df

In [ ]:
Image(filename=output_interbp_correlation_jpg_path,width = 600)

<a id="intrahelparcorrelation"></a>
### Helical Parameter Correlations: Intra-base pair 

In [ ]:
from biobb_dna.intrabp_correlations.intrahpcorr import intrahpcorr

timeseries_shear = timeseries_dir+"/series_shear_"+timesel.value[:-1]+".csv"
timeseries_stretch = timeseries_dir+"/series_stretch_"+timesel.value[:-1]+".csv"
timeseries_stagger = timeseries_dir+"/series_stagger_"+timesel.value[:-1]+".csv"
timeseries_buckle = timeseries_dir+"/series_buckle_"+timesel.value[:-1]+".csv"
timeseries_propel = timeseries_dir+"/series_propel_"+timesel.value[:-1]+".csv"
timeseries_opening = timeseries_dir+"/series_opening_"+timesel.value[:-1]+".csv"

output_helpar_bp_correlation_csv_path = "helpar_bp_correlation.csv"
output_helpar_bp_correlation_jpg_path = "helpar_bp_correlation.jpg"

intrahpcorr(
    input_filename_shear=timeseries_shear,
    input_filename_stretch=timeseries_stretch,
    input_filename_stagger=timeseries_stagger,
    input_filename_buckle=timeseries_buckle,
    input_filename_propel=timeseries_propel,
    input_filename_opening=timeseries_opening,
    output_csv_path=output_helpar_bp_correlation_csv_path,
    output_jpg_path=output_helpar_bp_correlation_jpg_path)


In [ ]:
df = pd.read_csv(output_helpar_bp_correlation_csv_path)
df

In [ ]:
Image(filename=output_helpar_bp_correlation_jpg_path,width = 600)

<a id="interhelparcorrelation"></a>
### Helical Parameter Correlations: Inter-base pair steps

In [ ]:
from biobb_dna.interbp_correlations.interhpcorr import interhpcorr

timeseries_shift = timeseries_dir+"/series_shift_"+timesel.value+".csv"
timeseries_slide = timeseries_dir+"/series_slide_"+timesel.value+".csv"
timeseries_rise = timeseries_dir+"/series_rise_"+timesel.value+".csv"
timeseries_tilt = timeseries_dir+"/series_tilt_"+timesel.value+".csv"
timeseries_roll = timeseries_dir+"/series_roll_"+timesel.value+".csv"
timeseries_twist = timeseries_dir+"/series_twist_"+timesel.value+".csv"

output_helpar_bps_correlation_csv_path = "helpar_bps_correlation.csv"
output_helpar_bps_correlation_jpg_path = "helpar_bps_correlation.jpg"

interhpcorr(
    input_filename_shift=timeseries_shift,
    input_filename_slide=timeseries_slide,
    input_filename_rise=timeseries_rise,
    input_filename_tilt=timeseries_tilt,
    input_filename_roll=timeseries_roll,
    input_filename_twist=timeseries_twist,
    output_csv_path=output_helpar_bps_correlation_csv_path,
    output_jpg_path=output_helpar_bps_correlation_jpg_path)


In [ ]:
df = pd.read_csv(output_helpar_bps_correlation_csv_path)
df

In [ ]:
Image(filename=output_helpar_bps_correlation_jpg_path,width = 600)

<a id="intrabasepaircorrelation"></a>
### Neighboring steps Correlations: Intra-base pair 

In [ ]:
from biobb_dna.intrabp_correlations.intrabpcorr import intrabpcorr

canal_shear = canal_dir+"/canal_output_shear.ser"
canal_stretch = canal_dir+"/canal_output_stretch.ser"
canal_stagger = canal_dir+"/canal_output_stagger.ser"
canal_buckle = canal_dir+"/canal_output_buckle.ser"
canal_propel = canal_dir+"/canal_output_propel.ser"
canal_opening = canal_dir+"/canal_output_opening.ser"

output_bp_correlation_csv_path = "bp_correlation.csv"
output_bp_correlation_jpg_path = "bp_correlation.jpg"

prop = {
    'sequence' : seq
}

intrabpcorr(
    input_filename_shear=canal_shear,
    input_filename_stretch=canal_stretch,
    input_filename_stagger=canal_stagger,
    input_filename_buckle=canal_buckle,
    input_filename_propel=canal_propel,
    input_filename_opening=canal_opening,
    output_csv_path=output_bp_correlation_csv_path,
    output_jpg_path=output_bp_correlation_jpg_path,
    properties=prop)

In [ ]:
df = pd.read_csv(output_bp_correlation_csv_path)
df

In [ ]:
Image(filename=output_bp_correlation_jpg_path,width = 800)

<a id="interbasepaircorrelation"></a>
### Neighboring steps Correlations: Inter-base pair steps

In [ ]:
from biobb_dna.interbp_correlations.interbpcorr import interbpcorr

canal_shift = canal_dir+"/canal_output_shift.ser"
canal_slide = canal_dir+"/canal_output_slide.ser"
canal_rise = canal_dir+"/canal_output_rise.ser"
canal_tilt = canal_dir+"/canal_output_tilt.ser"
canal_roll = canal_dir+"/canal_output_roll.ser"
canal_twist = canal_dir+"/canal_output_twist.ser"

output_bps_correlation_csv_path = "bps_correlation.csv"
output_bps_correlation_jpg_path = "bps_correlation.jpg"

prop = {
    'sequence' : seq
}

interbpcorr(
    input_filename_shift=canal_shift,
    input_filename_slide=canal_slide,
    input_filename_rise=canal_rise,
    input_filename_tilt=canal_tilt,
    input_filename_roll=canal_roll,
    input_filename_twist=canal_twist,
    output_csv_path=output_bps_correlation_csv_path,
    output_jpg_path=output_bps_correlation_jpg_path,
    properties=prop)

In [ ]:
df = pd.read_csv(output_bps_correlation_csv_path)
df

In [ ]:
Image(filename=output_bps_correlation_jpg_path,width = 800)

***
<a id="questions"></a>

## Questions & Comments

Questions, issues, suggestions and comments are really welcome!

* GitHub issues:
    * [https://github.com/bioexcel/biobb](https://github.com/bioexcel/biobb)

* BioExcel forum:
    * [https://ask.bioexcel.eu/c/BioExcel-Building-Blocks-library](https://ask.bioexcel.eu/c/BioExcel-Building-Blocks-library)
